In [ ]:
import os
import subprocess
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from git import Repo
import seaborn as sns
import matplotlib.pyplot as plt

def get_commit_data(repo_path, branch_name):
    repo = Repo(repo_path)
    commit_data = []

    for commit in repo.iter_commits(branch_name):
        commit_date = datetime.fromtimestamp(commit.committed_date)
        if commit_date > datetime.now() - timedelta(days=365):
            commit_stats = commit.stats.total
            commit_data.append({
                'author': commit.author.name,
                'date': commit_date,
                'lines_added': commit_stats['insertions']
            })
    return commit_data

def analyze_commit_data(commit_data):
    df = pd.DataFrame(commit_data)
    df['day'] = df['date'].dt.date
    df['week'] = df['date'].dt.to_period('W')
    df['date'] = pd.to_datetime(df['date'])  # Convert 'date' column to datetime type
    pd.set_option('display.max_rows', None)
    min_date = df['date'].min()
    df['30_day_interval'] = ((df['date'] - min_date).dt.days // 30).astype(int)  # Calculate 30-day interval

    # Calculate lines added per developer in each week
    pivot_table = pd.pivot_table(df, values='lines_added', index='author', columns='30_day_interval', aggfunc='sum')
    pivot_table.columns = ['Days_' + str(col * 30) + '-' + str((col+1) * 30) for col in pivot_table.columns]  # Rename column headers
    # Calculate the average lines added across all developers
    avg_lines_added = pivot_table.mean(axis=0)

    # Add a final column to the pivot table that sums the values of each row
    pivot_table['Total'] = pivot_table.sum(axis=1)
    
    return pivot_table, avg_lines_added


if __name__ == "__main__":
    repo_path = input("Enter the path to the local git repository: ")
    branch_name = input("Enter the branch name: ")

    commit_data = get_commit_data(repo_path, branch_name)
    result, avg_lines_added = analyze_commit_data(commit_data)

    pd.set_option('display.max_columns', None)
#     print(result)

    # Create a styled DataFrame with a background gradient
    result_added = result.style.background_gradient(cmap='YlOrRd')

    # Display the styled DataFrame
    display(result_added)


In [ ]:
import qgrid

In [ ]:

qgrid_widget = qgrid.show_grid(result)
display(qgrid_widget)